<a href="https://colab.research.google.com/github/dha5sm/project_voting/blob/main/Write_up_project_voiting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!git clone https://github.com/dha5sm/project_voting/

import pandas as pd
import numpy as np
import os
from functools import reduce
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

fatal: destination path 'project_voting' already exists and is not an empty directory.


**Summary**

FINISH

**Data**

To prepare the data for analysis we first scanned over the csv files contained within the “county_data” folder. We filtered out the demographic estimates within this folder to include only those from the years 2016 - 2020. Subsequently, we then worked to combine these separate dataframes into one main demographics data frame using the ‘GISJOIN’ field. This data table included all the demographic data for each county over the time period. We then filtered this data further to include only counties with the state code VA. The result was one demographics dataframe that contained data on every county in Virginia.


Next, we worked to filter all the demographic data, selecting key fields in each county related to: Race, median household income, education status, and the frequency with which different languages are spoken at home. In consultation with the codebook, the variable names were identified and cleaned to have new, more easily understandable names.



In [11]:
#Getting the name of all csv files to use
direct = './project_voting/data/county_data'
files = []
for file in os.listdir(direct):
  if file.endswith("E.csv"):
    files.append(file)
dataframes = []
for file in files:
  if file == "0002_ds250_20205_county_E.csv" or file == "0002_ds249_20205_county_E.csv":
    df_cur = pd.read_csv(direct + "/" + file, encoding='ISO-8859-1')

    df_for_va =  df_cur[df_cur['STATE'] == 'Virginia']

    dataframes.append(df_for_va)

<ipython-input-11-4d18b20433f5>:10: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266

In [12]:
#Only using most recent data of VA demographics to keep managable from 2016 to 2020
columns_keep = dataframes[0].columns
columns_keep = columns_keep.drop('GISJOIN')

df2 = dataframes[1]

for col in columns_keep:
    if col in df2:
        df2 = df2.drop(columns=col)

df_VA2020 = pd.merge(dataframes[0], df2, on='GISJOIN')

#df_VA2020 contains all the demographic information for Virginia on a county by county basis for the years 2016 - 2020
#There are some columns with NAN values but will filter based on which columns are used / looked at

With that table in hand, we then worked to create a table with all the voting data by county in Virginia for the most recent election. This involved taking the original voting table and filtering by the most recent election. Because the same democratic and republican candidate are up for election, we determined this to be the most relevant years.


In [14]:
#Get Data and check NA count
df_Votes = pd.read_csv("./project_voting/data/voting_VA.csv")
df_Votes.shape
na_count = df_Votes.isna().sum()

df_Votes_2020 = df_Votes[df_Votes['year'] == 2020]

df_Votes_party = df_Votes_2020.pivot_table(index="county_name", columns="party", values="candidatevotes", fill_value=0).reset_index()
df_Votes_party.head(20)

df_Votes_party.shape

(133, 5)

Next, for each county in the table there were 4 rows that displayed vote count by 4 parties: Democrat, Republican, Green, and Other. We believed this data would be more usable if for every county in Virginia there was one single row, instead of four, that contained a column showcasing how many votes each party received. The pivot table operation was used to accomplish this task.


In [16]:
cols = ['DEMOCRAT','LIBERTARIAN', 'OTHER', 'REPUBLICAN' ]

def convert_str(val):
  float_n = float(val)
  int_n = int(float_n)
  return int_n

for col in cols:
  df_Votes_party[col] = df_Votes_party[col].apply(convert_str)

#https://saturncloud.io/blog/how-to-sum-values-of-a-row-of-a-pandas-dataframe-efficiently/#:~:text=The%20most%20efficient%20way%20to,the%20values%20in%20each%20row.
df_Votes_party['total_votes'] = df_Votes_party[ cols ].sum(axis = 1)

#Check the data appears as expected
df_Votes_party.head()

party,county_name,DEMOCRAT,LIBERTARIAN,OTHER,REPUBLICAN,total_votes
0,ACCOMACK,2526,62,8,3057,5653
1,ALBEMARLE,14155,338,124,6934,21551
2,ALEXANDRIA CITY,22080,340,238,4848,27506
3,ALLEGHANY,747,29,4,1953,2733
4,AMELIA,803,26,4,1796,2629


Following this, we noticed that some counties had a decimal figure for how many people voted for a party (probably due to the fact the original data was an estimate of the population). As such, we decided to convert the counties from strings to float values, after which we truncated the values by casting them to the integer type. Because the vote counts are so, losing a fraction of a voter would have limited impact on any predictions. Lastly, the various vote counts were then summed to create a new total count column.


In [20]:
cols_to_eval = []
df_test = df_VA2020[:20]

#Keeping the relevant information
columns_keep = ['GISJOIN', 'YEAR', 'STATE', 'COUNTY', 'COUNTYA', 'AMPWE001', 'AMPWE002', 'AMPWE003', 'AMPWE004', 'AMPWE005', 'AMPWE006', 'AMPWE007', 'AMR8E001', 'AMRZE022', 'AMRZE001', 'AMZLE001', 'AMZLE002', 'AMZLE003']
df_select_data = df_VA2020[columns_keep]

#Renaming the variables for analysis
col_names = {
    'AMPWE001':'Total_Reporting_Race',
    'AMPWE002':'White',
    'AMPWE003':'African_American',
    'AMPWE004': 'Indian_or_Alaska',
    'AMPWE005': 'Asian',
    'AMPWE006' : 'Pacific_Islander',
    'AMPWE007' : 'Other',
    'AMR8E001': 'Median_House_Income',
    'AMRZE022': 'People_with_Bachelors',
    'AMRZE001': 'Total_pop_asked_education',
    'AMZLE001' : 'Total_reporting_lang_at_home',
    'AMZLE002': 'English_in_home',
    'AMZLE003': 'Spanish_in_home'
}

df_select_data.rename(columns= col_names , inplace=True)
df_select_data['Other_lang_at_home'] = df_select_data['Total_reporting_lang_at_home'] - df_select_data['English_in_home'] - df_select_data['Spanish_in_home']


<ipython-input-20-477b406f7d5d>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select_data.rename(columns= col_names , inplace=True)
<ipython-input-20-477b406f7d5d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select_data['Other_lang_at_home'] = df_select_data['Total_reporting_lang_at_home'] - df_select_data['English_in_home'] - df_select_data['Spanish_in_home']


Finally, we max - min adjusted the variables to that we could run a knn model and get our predictions.

In [25]:
df_Votes_party['net_votes'] = df_Votes_party['DEMOCRAT'] - df_Votes_party['REPUBLICAN']
def max_min_normalization(column):
    """Apply max-min normalization to a Pandas Series."""
    return (column - column.min()) / (column.max() - column.min())

The final form of the data prior to the analysis is highlighted in the table below.

In [26]:
df_select_data['county_name'] = df_select_data['COUNTY'].str.upper().str.replace(' COUNTY', '', regex=False)

#Merging df_votes and df_select on the 'county_name' column
df_merged = pd.merge(df_Votes_party, df_select_data, on='county_name', how='inner')

# Displaying merged df
df_merged.head()

<ipython-input-26-a9ec6b6e9577>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select_data['county_name'] = df_select_data['COUNTY'].str.upper().str.replace(' COUNTY', '', regex=False)


,county_name,DEMOCRAT,LIBERTARIAN,OTHER,REPUBLICAN,total_votes,net_votes,GISJOIN,YEAR,STATE,...,Asian,Pacific_Islander,Other,Median_House_Income,People_with_Bachelors,Total_pop_asked_education,Total_reporting_lang_at_home,English_in_home,Spanish_in_home,Other_lang_at_home
0,ACCOMACK,2526,62,8,3057,5653,-531,G5100010,2016-2020,Virginia,...,244,0,154,46178.0,2697,23813,13641,12355,679,607
1,ALBEMARLE,14155,338,124,6934,21551,7221,G5100030,2016-2020,Virginia,...,5981,48,989,84643.0,22495,73818,42381,37126,2058,3197
2,ALEXANDRIA CITY,22080,340,238,4848,27506,17232,G5105100,2016-2020,Virginia,...,10588,91,8565,102227.0,37466,119957,71289,50347,7869,13073
3,ALLEGHANY,747,29,4,1953,2733,-1206,G5100050,2016-2020,Virginia,...,10,0,64,48513.0,943,11155,6529,6376,89,64
4,AMELIA,803,26,4,1796,2629,-993,G5100070,2016-2020,Virginia,...,1,0,0,63918.0,1046,9270,5034,4799,206,29


**Results**

To predict the election within the state of virginia for 2024, we decided to go with a KNN model. Specifically using the variables like Median household income and people with bachelors we were able to predict the outcome more accurately.  

In [27]:
# Normalizing the variables to use in the KNN model
features_to_normalize = ['Median_House_Income', 'Total_Reporting_Race', 'People_with_Bachelors']
for feature in features_to_normalize:
    df_merged[feature] = max_min_normalization(df_merged[feature])

In [28]:
X_normalized = df_merged[features_to_normalize].values
y = df_merged['net_votes'].values

# Splitting the data
X_train_norm, X_test_norm, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [29]:
def evaluate_k_values(X, y, k_values):
    mse_scores = []
    for k in k_values:
        knn = KNeighborsRegressor(n_neighbors=k)
        scores = cross_val_score(knn, X, y, scoring='neg_mean_squared_error', cv=5)
        mse_scores.append(np.mean(np.abs(scores)))  # Converting scores to positive MSE values
    return mse_scores

With the KNN model, we had to find the optimal k-value to reduce the mean squared error in the model without overfitting it to the test data.

In [30]:
k_values = range(1, 31)

# Evaluating MSE for each k-value
mse_scores = evaluate_k_values(X_train_norm, y_train, k_values)

# Finding the optimal k with the lowest MSE
optimal_k = k_values[np.argmin(mse_scores)]
optimal_k, mse_scores

(2,
 [59202641.22164502,
  54184041.16417749,
  62860008.5948052,
  63412320.93457793,
  68400672.67960171,
  72880615.58252767,
  71255811.77657919,
  69977767.74191694,
  73224200.19723691,
  74022147.21366233,
  75529345.63860326,
  77195206.65926827,
  79291678.58433105,
  80016579.06311955,
  81466398.70196827,
  82957986.78797181,
  83730857.20088378,
  83757387.28866042,
  84830152.84600016,
  85604865.31973055,
  86654150.77252701,
  87536887.31503256,
  88504332.96080819,
  89055671.93304096,
  89698483.69637887,
  90112676.32630959,
  90387387.35187264,
  90165434.9802307,
  90425648.54297142,
  90166488.2209091])

With this we can run our KNN model. Using our training data, we found that we can predict whether a country will be democratic or republican with a 74% accuracy rate.


In [40]:
#labels 1 for democrat and 0 for republican
df_merged['actual_label'] = (df_merged['net_votes'] > 0).astype(int)

# verifying optimal k value
knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train_norm, y_train)

# Finding the predicted net votes for the test set
predicted_net_votes = knn.predict(X_test_norm)

# Convert the predictions back into binary codes
predicted_labels = (predicted_net_votes > 0).astype(int)

actual_labels = (y_test > 0).astype(int)

# Count correct and incorrect predictions
correct_predictions = np.sum(predicted_labels == actual_labels)
incorrect_predictions = len(predicted_labels) - correct_predictions

correct_predictions, incorrect_predictions
print("Accuracy: ", (correct_predictions)/(correct_predictions + incorrect_predictions))

Accuracy:  0.7407407407407407


It must be noted here that the data used was from the 2020 elections. It does not completly represent what will occur in 2024. More generally we know that people's opinions are becoming more and more polarized by party and there is less switching occuring by candidate.

Further analyzing our results with a confusion matrix we can see the 15/20 of our actual voting republicans, were predicted as such. Furthermore, 5/7 of our actual democrats were predicted as such. This 75% and 71% accuracy further highlights the ability of the model to use people's demographics to correctly identify the overall county level vote.

However, there is still some areas for improvement as the 7 counties were predicted wrong out of just 27. This would be enough to sway an election or the overall vote of a state. Further highlights how we can't be fully reliant on the output of this model's predictions.

In [45]:
# Generate statistics
print("Classification Report:")
print(classification_report(actual_labels, predicted_labels))

# Show confusion matrix
cm = confusion_matrix(actual_labels, predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Republican', 'Democrat'], yticklabels=['Republican', 'Democrat'])
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')
plt.title('Confusion Matrix')
plt.show()

Classification Report:


NameError: name 'classification_report' is not defined

**Conclusion**

FINISH